In [ ]:
################################################################################### 사용 library
import pandas as pd
import numpy as np
import os
import openpyxl

######################################################################## 1. 통합 월매출데이터 생성 

# sheet name 가져오기
wb = openpyxl.load_workbook("월별매출데이터.xlsx")
sheet_names = wb.sheetnames

# data 통합
df = pd.DataFrame()

for sheet_name in sheet_names:
    df = pd.concat([df,pd.read_excel("월별매출데이터.xlsx", 
                                     sheet_name= sheet_name,
                                     skiprows=range(6),
                                     usecols="B:I")], axis = 0)
    
# 포맷 통일 
df['일자'] = pd.to_datetime(df['일자'])

# 제품가격정보
df2 = pd.read_excel("제품별_가격정보.xlsx")

# 주문 금액 
dict1 = pd.DataFrame(df2.T[1:])[0].to_dict()
df['주문 금액'] = df['품명'].replace(dict1) * df['수량'] * 1.1

# 주소 데이터 분리
df = pd.concat([df.drop('수령 주소', axis = 1),
                df['수령 주소'].str.split(' ',expand = True)], 
               axis = 1)

df.rename({0 : '시',
           1 : '구',
           2 : '동(가)'}, axis = 1, inplace = True)

# 통합 월매출데이터 완성


############################################################################# 2. 고객 데이터 생성 

# 총 구매액 - 구매력을 측정하기 위한 변수
customer_df = df.groupby('주문인 ID', as_index = False)['주문 금액'].agg(['sum','count'])

# 30일당 구매 수 - 구매 빈도를 측정하기 위한 변수 
customer_df['기간 구매율'] = customer_df['count'] / 365

customer_df.drop('count', axis = 1, inplace = True)

customer_df['기간 구매율'] = customer_df['기간 구매율'] * 30

customer_df.rename({'기간 구매율' : '30일당 구매수',
                    'sum' : '총 구매액'}, axis = 1, inplace = True)

# 변수생성
# - 최근성 : 이탈 여부를 확인하기 위한 변수
# - 고객유지일 수 : 사용 기간을 측정하기 위한 변수
df.sort_values(by = ['일자', '주문인 ID'],inplace=True) # 첫(마지막) 거래일을 추출하기 위한 정렬

pre_df = df.drop_duplicates(['주문인 ID'], keep='last').loc[:,['주문인 ID', '일자']] # 임시 df

pre_df['마지막 구매일'] = pd.to_datetime(pre_df['일자'])
pre_df['최근성'] = (pd.to_datetime('20181231') - pre_df['일자']).apply(str).str.split(' ', expand = True).iloc[:,0]
pre_df['첫 구매일'] = pre_df['주문인 ID'].replace(df.drop_duplicates(['주문인 ID'], keep='first').loc[:,['주문인 ID', '일자']].set_index('주문인 ID')['일자'].to_dict())
pre_df['고객유지일 수'] = (pre_df['마지막 구매일'] - pre_df['첫 구매일']).apply(str).str.split(' ',expand = True).iloc[:,0]

pre_df.drop('일자', axis =1, inplace = True)

pre_df.set_index('주문인 ID', inplace = True) # merge를 위한 setting

customer_df = customer_df.merge(pre_df, left_index=True, right_index=True)

customer_df['최근성'] = customer_df['최근성'].astype(int) 
customer_df['고객유지일 수'] = customer_df['고객유지일 수'].astype(int) 

# outlier criteria - 고객별 거래일 간 간격을 고려한 이탈 고객 기준 변수
# -> 고객 구매일 딕셔너리 : 고객 별 구매 일을 저장한 딕셔너리
pre_df = df[['주문인 ID', '일자']]
pre_df = pre_df.drop_duplicates(['주문인 ID', '일자'], keep = 'first')

pre_dict = dict()
for id_ in pre_df['주문인 ID'].unique(): # 고객 별 구매일 key 설정 
    
    pre_dict[id_] = []

for id_, date in pre_df.values : # 고객 별 구매일 삽입
    
     pre_dict[id_] = pre_dict[id_] + [date]
        
pre_dict2 = dict()
for id_ in pre_dict.keys() : # 고객 별 구매일 간격 계산
    
    pre_dict2[id_] = []
    
    for  i in range(1,len(pre_dict[id_])) :
        
        pre_dict2[id_] = pre_dict2[id_] + [pre_dict[id_][i] - pre_dict[id_][i-1]]
        
    pre_dict2[id_] = [int(str(i).split(' ')[0]) for i in pre_dict2[id_]]
    
pre_dict3 = dict()
for id_ in pre_dict2.keys(): # iqr 룰을 통한 고객별 이탈고객 criteria 계산
    
    pre_list = pre_dict2[id_]
    q3 = np.quantile(pre_list, 0.75)
    q1 = np.quantile(pre_list, 0.25)
    iqr = q3-q1
    criteria = q3 + 1.5*iqr
    pre_dict3[id_] = criteria

customer_df['outlier criteria'] = pd.Series(pre_dict3)

# 이탈고객 - iqr 룰을 통해 만든 outlier criteria를 기준으로한 이탈고객
customer_df['이탈고객'] = customer_df['최근성'] > customer_df['outlier criteria']

# 우수고객 - 이탈고객이 아니면서 구매력, 구매빈도, 사용 기간 세가지 지표 모두 상위 10% 안에 속하는 고객
# -> 4명으로 약 1%
customer_df['우수고객'] = pd.Series((customer_df['총 구매액'] >= customer_df['총 구매액'].quantile(0.9)) & (customer_df['30일당 구매수'] >= customer_df['30일당 구매수'].quantile(0.9)) & (customer_df['고객유지일 수'] >= customer_df['고객유지일 수'].quantile(0.9)) & ~customer_df['이탈고객'],
                                   index=customer_df.index)

# 수령받은 구 개수, 구매 물품 수, 이용 건 수 - 고객 특성을 파악하기 위해 생성
customer_df['수령받은 구 개수'] = pd.crosstab(index = df['주문인 ID'], 
                                             columns = df['구'] ).apply(lambda x : sum(x>0),
                                                                        axis = 1)
customer_df['구매 물품 수'] = df.groupby('주문인 ID')['수량'].sum()
customer_df['이용 건 수'] = df['주문인 ID'].value_counts()

# 시각화에 사용할 고객유형 변수 생성
customer_df['고객유형'] = [1]*customer_df.shape[0]
customer_df.loc[customer_df['우수고객'] == 1 ,'고객유형'] = '우수고객'
customer_df.loc[customer_df['이탈고객'] == 1 ,'고객유형'] = '이탈고객'
customer_df.loc[(customer_df['우수고객'] != 1) & (customer_df['이탈고객'] != 1) ,'고객유형'] = '일반고객'



############################################################################### 3. 구 데이터 생성 

gu_df = df.groupby('구', as_index = False)['주문 금액'].sum().sort_values('주문 금액')
gu_df.set_index('구', inplace = True)

# 구별 고객 수, 1인당 구매 금액, 주문 건수 : 구별 특징을 파악하기 위해 생성
gu_df['구별 고객 수'] = df[['구', '주문인 ID']].drop_duplicates(['구', '주문인 ID']).groupby('구')['주문인 ID'].count()
gu_df['1인당 구매 금액'] = gu_df['주문 금액'] / gu_df['구별 고객 수']
gu_df['주문 건수'] = df['구'].value_counts()
gu_df['총 판매량'] = gu_df.iloc[:,4:].apply(sum, axis = 1)


############################################################################# 4. 지점 데이터 생성

point_df = pd.DataFrame(df.groupby(['지점'])['품명'].count())
point_df['고객 수'] = df[['지점','주문인 ID']].drop_duplicates(['지점','주문인 ID']).groupby(['지점'])['주문인 ID'].count()
point_df = pd.concat([point_df, df.groupby(['지점','품명'])['수량'].sum().unstack(1)], axis = 1)
point_df = point_df.rename({'품명' : '배달 건수'}, axis = 1)
point_df = pd.concat([point_df,df.groupby(['지점','구'])['구'].count().unstack(1)], axis = 1)
point_df['총 제품 판매량'] = point_df.iloc[:,2:8].apply(sum,axis = 1)
point_df.rename({'품명' : '배달 건수'},axis = 1, inplace='True')

################################################################################### 5. 엑셀 저장

writer = pd.ExcelWriter('통합 매출데이터.xlsx', engine = 'openpyxl')

df.to_excel(writer, sheet_name = 'df')
customer_df.to_excel(writer, sheet_name = 'customer_df')
gu_df.to_excel(writer, sheet_name = 'gu_df')
point_df.to_excel(writer, sheet_name = 'point_df')

writer.save()